In [120]:
import os  
from dotenv import load_dotenv 

from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools import YouTubeSearchTool
from langchain_community.tools.tavily_search import TavilySearchResults

# Custom Tools
from langchain.tools import tool

# Agents
from langchain.agents import AgentType, load_tools, initialize_agent, create_tool_calling_agent, AgentExecutor
from langchain import hub # Import the hub module to access pre-built prompts and agents from LangChain's repository

from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.tools.retriever import create_retriever_tool

from langchain.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool, create_react_agent, create_self_ask_with_search_agent

load_dotenv()

True

In [121]:
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_PROJECT = os.getenv("LANGCHAIN_PROJECT")

In [122]:
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_PROJECT"] = LANGCHAIN_PROJECT
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"

In [123]:
llm = ChatGroq(model_name="Gemma2-9b-It")
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

Predefine tools

In [124]:
api_wrapper = WikipediaAPIWrapper()
tool1 = WikipediaQueryRun(api_wrapper=api_wrapper)
print(tool1.invoke({"query":"langchain"}))

Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.

Page: Retrieval-augmented generation
Summary: Retrieval-augmented generation (RAG) is a technique that enables generative artificial intelligence (Gen AI) models to retrieve and incorporate new information. It modifies interactions with a large language model (LLM) so that the model responds to user queries with reference to a specified set of documents, using this information to supplement information from its pre-existing training data. This allows LLMs to use domain-specific and/or updated information. Use cases include providing chatbot access to internal company data or generating responses based on authoritative sources.
RAG improves larg

In [125]:
tool2 = YouTubeSearchTool()
tool2.run({"query":"sidemen"})

"['https://www.youtube.com/watch?v=mFa13CIo2No&pp=ygUHc2lkZW1lbg%3D%3D', 'https://www.youtube.com/watch?v=yGlRj2Lh_V8&pp=ygUHc2lkZW1lbg%3D%3D']"

In [126]:
tool3 = TavilySearchResults()
tool3.run("Bankai")

[{'title': 'Bankai - Bleach Untold Stories Wiki - Fandom',
  'url': 'https://bleachuntoldstories.fandom.com/wiki/Bankai',
  'content': 'Bankai\n\nThe\xa0Bankai\xa0(卍解,\xa0Final Release) is the second and final upgraded form of a Zanpakutō. To achieve Bankai, one must be able to materialize and subjugate their Zanpakutō spirit. Materialization means the opposite of getting dragged into the Zanpakutō\'s inner world. Instead the wielder needs to summon the Zanpakutō\'s spirit into the physical world. It usually takes 10 years or more to achieve, plus the experience needed to master it. [...] Bankai Forms: After a Shinigami is able to materialize and subjugate their Zanpakutō\'s spirit, the Zanpakutō can assume its final and most powerful form. Unlike the Shikai, there is no command phrase to activate Bankai, though the user will usually say "Bankai" beforehand. A Zanpakutō\'s Bankai is often a manifestation of the spirit itself, usually creating a massive creature or a very powerful effec

Custom Tool

In [127]:
@tool
def get_word_len(word : str) -> int:
    """Returns the length of word."""
    return len(word)

In [128]:
get_word_len.invoke("abc")

3

In [129]:
@tool
def multiply(a : int, b : int) -> int:
    """Multiply two numbers."""
    return a * b

In [130]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [131]:
multiply.invoke({"a": 4, "b" : 5})

20

# Concepts of Agents

This agent class from the first version of langchain (legacy)

In [132]:
tool1 = load_tools(['wikipedia'],llm=llm)

In [133]:
agent = initialize_agent(
    tools=tool1,
    llm = llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

In [134]:
agent.run("What's the icc world cup stats of India?")



> Entering new AgentExecutor chain...
Thought: I need to know India's performance in the ICC World Cup tournaments. 
Action: wikipedia
Action Input: ICC Cricket World Cup results
Observation: Page: 2011 Cricket World Cup
Summary: The 2011 ICC Cricket World Cup was the tenth Cricket World Cup. It was played in India, Sri Lanka and Bangladesh, while the latter hosted World Cup matches for the first time. India won the tournament, defeating Sri Lanka by six wickets in the final at Wankhede Stadium in Mumbai, thus becoming the first country to win the Cricket World Cup final on home soil. India's Yuvraj Singh was declared as the player of the tournament. This was the first time in World Cup history that two Asian teams had appeared in the final. It was also the first time since the 1992 World Cup that the final did not feature Australia.
Fourteen national cricket teams took part in this tournament, including 10 full members and four associate members of the International Cricket Council 

'India has won the ICC Cricket World Cup twice: in 1983 and 2011.'

In [135]:
search = [TavilySearchResults()]

In [136]:
prompt = hub.pull("hwchase17/openai-functions-agent")

In [137]:
prompt # agent scratchpad for repeated calling of tool

ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='

In [138]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [139]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=search,
    prompt=prompt
)

In [140]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=search,
    verbose=True
)

In [141]:
agent_executor.invoke({"input":"Whats the weather in mumbai"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in mumbai'}`


[{'title': "Mumbai - India weather forecast's post - Facebook", 'url': 'https://www.facebook.com/meteobox.mumbai/posts/mumbai-thursday-10042025-weather-forecast-for-bombay-april-10-2025during-the-day/1220986733365604/', 'content': '#Mumbai, Thursday 10/04/2025: Weather forecast for Bombay April 10, 2025During the day clear. The highest temperature 34 °C . At night', 'score': 0.9338243}, {'title': 'Weather in mumbai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1744282474, 'localtime': '2025-04-10 16:24'}, 'current': {'last_updated_epoch': 1744281900, 'last_updated': '2025-04-10 16:15', 'temp_c': 31.1, 'temp_f': 88.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png

{'input': 'Whats the weather in mumbai',
 'output': '\n\n\n}\n\n\n\n\n\n\n\n```json\n```\n"\n"\n\n```\n\n```\n```\n\n```\n\n```\n```\n\n```\n```\n```\n\n```\n```\n"\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n```\n``

In [142]:
agent_executor.invoke({"input":"Hey, how are you ?"})



> Entering new AgentExecutor chain...
I'm doing well, thank you! How can I help you today?

> Finished chain.


{'input': 'Hey, how are you ?',
 'output': "I'm doing well, thank you! How can I help you today?"}

unnecessary use of tool can be seen in above example, hence langgraph will come in picture

1. RAG tool
2. ReAct
3. Custom tool with ReAct
4. agent code from latest ver v2 and v3

<img src="../assets/Screenshot 2025-04-10 134338.png">

In [143]:
loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs  = loader.load()

In [144]:
docs

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content="\n\n\n\n\nGet started with LangSmith | 🦜️🛠️ LangSmith\n\n\n\n\n\n\nSkip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn this pageGet started with LangSmith\nLangSmith is a platform for building production-grade LLM applications.\nIt allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.\nObservabilityAn

In [145]:
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=400
).split_documents(docs)

In [146]:
documents

[Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith'),
 Document(metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Skip to main contentJoin us at  Interrupt: The Agent AI Conference by LangChain on May 13 & 14 in San Francisco!API ReferenceRESTPythonJS/TSSearchRegionUSEUGo to AppGet StartedObservabilityEvaluationPrompt EngineeringDeployment (LangGraph Platform)AdministrationSelf-hostingPricingReferenceCloud architecture and scalabilityAuthz and AuthnAuthentication methodsdata_formatsEvaluationDataset transformationsRegions FAQsdk_referenceGet StartedOn t

In [147]:
vector = FAISS.from_documents(documents=documents,embedding=embeddings)
retriever = vector.as_retriever()

In [148]:
retriever

VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002780D0CCD50>, search_kwargs={})

In [149]:
retriever.invoke("What is langsmith ?")[0]

Document(id='bad9ee15-ce19-4f69-b3fa-bc8913cafd11', metadata={'source': 'https://docs.smith.langchain.com/overview', 'title': 'Get started with LangSmith | 🦜️🛠️ LangSmith', 'description': 'LangSmith is a platform for building production-grade LLM applications.', 'language': 'en'}, page_content='Get started with LangSmith | 🦜️🛠️ LangSmith')

In [150]:
retriever_tool  = create_retriever_tool(
    retriever,
    "langsmith_studio",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!"
)

In [151]:
tools = [tool3,retriever_tool]

In [152]:
agent = create_tool_calling_agent(llm,tools,prompt)

In [153]:
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True
)

In [154]:
agent_executor.invoke({"input":"Hi, whats Langsmith and how to use it?"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_studio` with `{'query': 'what is Langsmith and how to use it'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

For more see the how-to guide for setting up LangSmith with LangChain or setting up LangSmith with LangGraph.
Observability​
Observability is important for any software application, but especially so for LLM applications. LLMs are non-deterministic by nature, meaning they can produce unexpected results. This makes them trickier than normal to debug.
This is where LangSmith can help! LangSmith has LLM-native observability, allowing you to get meaningful insights from your application. LangSmith’s observability features have you covered throughout all stages of application development - from prototyping, to beta testing, to production.

ObservabilityAnalyze traces in LangSmith and configure metrics, dashboards, alerts based on these.EvalsEvaluate your application over production traffic — score application performance

{'input': 'Hi, whats Langsmith and how to use it?',
 'output': 'LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence.  '}

In [155]:
agent_executor.invoke({"input":"whats weather in mumbai?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in mumbai'}`


[{'title': "Mumbai - India weather forecast's post - Facebook", 'url': 'https://www.facebook.com/meteobox.mumbai/posts/mumbai-thursday-10042025-weather-forecast-for-bombay-april-10-2025during-the-day/1220986733365604/', 'content': '#Mumbai, Thursday 10/04/2025: Weather forecast for Bombay April 10, 2025During the day clear. The highest temperature 34 °C . At night', 'score': 0.9338243}, {'title': 'Weather in mumbai', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Mumbai', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.975, 'lon': 72.826, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1744282474, 'localtime': '2025-04-10 16:24'}, 'current': {'last_updated_epoch': 1744281900, 'last_updated': '2025-04-10 16:15', 'temp_c': 31.1, 'temp_f': 88.0, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png

{'input': 'whats weather in mumbai?',
 'output': '```json\n{"\n  "score": \n  "content": \n  "title": \n  "url":\n  "description":\n  "parameter":\n  "properties":\n  "properties":\n  "parameters":\n  "parameters":\n  "parameters":\n  "parameters":\n  "properties":\n  "parameters":\n  "parameters":\n  "parameters":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":\n  "description":

ReAct Agent (Reasoning + Acting):
Definition:

The ReAct framework combines reasoning and acting in a single loop to handle tasks. The agent uses natural language reasoning (thinking through steps) and task actions (performing tasks like calculations or data retrieval).

The agent uses a combination of reasoning steps to guide actions in real-time, using feedback from those actions to further inform the next step in reasoning.

How it works:

- Step 1: The agent receives a question or task.
- Step 2: It reasons aloud (in natural language) about how to solve it.
- Step 3: Based on its reasoning, it takes actions (e.g., searching a database, calculating something).
- Step 4: The results of these actions are integrated into its reasoning and may trigger further actions.
- Step 5: It repeats the process until it arrives at a solution.

Key points:

- Combines thinking and doing (reasoning and actions).
- Performs iterative steps, updating its process based on action results.
- Typically handles complex decision-making scenarios.

Example:

- Original task: “Calculate the total number of apples in a basket if there are 4 baskets and 7 apples in each.”
- Reasoning: "I need to multiply 4 by 7 to get the total number of apples."
- Action: Perform the multiplication.
- Result: "There are 28 apples."
- Reasoning: "I am done."

In [156]:
google_search = GoogleSerperAPIWrapper()

In [157]:
tools = [
    Tool(
        name="Web Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [158]:
prompt = hub.pull("hwchase17/react")

In [159]:
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

In [160]:
search_agent = create_react_agent(llm,tools,prompt)

In [161]:
agent_executor = AgentExecutor(
    agent=search_agent,
    tools=tools,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [162]:
response = agent_executor.invoke({"input": "What was the main reason of causing the World War ?"})



> Entering new AgentExecutor chain...
Thought:  I need to research the main causes of World War I.
Action: Web Answer
Action Input: main causes of World War ISix Causes of World War I ___European Expansionism. ... ___Serbian Nationalism. ... ___The Assassination of Franz Ferdinand. ... ___Conflicts over Alliances. ... ___The Blank Check Assurance: Conspired Plans of Germany and Austria-Hungary. ... ___Germany Millenarianism – Spirit of 1914.Thought:  European Expansionism, Serbian Nationalism, and the Assassination of Franz Ferdinand were major contributing factors. 
Final Answer: There wasn't one single cause of World War I, but a complex interplay of factors. Some key reasons include European expansionism, rising Serbian nationalism, and the assassination of Archduke Franz Ferdinand. 




> Finished chain.


ReAct with Custom Tool

In [163]:
@tool
def get_employee_id(name):
    """To get employee id, it takes name as an arguments
    name(str): Name of the employee"""
    employees = {
        "Alice": "E001",
        "Bob": "E002",
        "Charlie": "E003",
        "Diana": "E004",
        "Evan": "E005",
        "Fiona": "E006",
        "George": "E007",
        "Hannah": "E008",
        "Ian": "E009",
        "Jasmine": "E010"
    }
    return employees.get(name, "Employee not found")

In [164]:
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
  }
  return employee_salaries.get(employee_id,"Employee not found")

In [165]:
tools = [get_employee_id,get_employee_salary] # instead of this hard-coded data we can provide our own db 

In [166]:
agent = create_react_agent(llm,tools,prompt)

In [167]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [168]:
agent_executor.invoke({"input":"Whats the salary of Jasmine?"})



> Entering new AgentExecutor chain...
Thought: I need to find Jasmine's employee ID first, then use that to get her salary.
Action: get_employee_id
Action Input: JasmineE010Thought: Now that I have Jasmine's employee ID, I can find her salary.
Action: get_employee_salary
Action Input: E01055000Thought: I now know the final answer
Final Answer: Jasmine's salary is 55000 


> Finished chain.


{'input': 'Whats the salary of Jasmine?',
 'output': "Jasmine's salary is 55000"}

Self-Ask with Search Agent:
Definition:

This approach allows the AI to ask itself follow-up questions when it doesn't immediately know the answer. It performs a recursive questioning process to break down complex queries into smaller, more manageable ones.
Once the AI generates these follow-up questions, it performs an external search (e.g., through Google or an internal database) to gather the necessary information before formulating a response.
How it works:

- Step 1: Receive a complex question.
- Step 2: The agent identifies sub-questions or follow-up questions.
- Step 3: It performs a search or fetches answers to these questions from external resources (like a web search).
- Step 4: The answers are aggregated to provide a complete response.

Key points:

- Emphasizes question decomposition.
- Relies on external search for sub-questions.
- Useful for answering open-ended or broad questions where the answer is not immediately available.

Example:

- Original question: “How many moons does Jupiter have?”
- Sub-question: “What is Jupiter?” (search)
- Sub-question: “What are moons?” (search)
- Finally, it retrieves the answer: "Jupiter has 79 moons."

In [169]:
tools = [
    Tool(
        name="Intermediate Answer",
        func=google_search.run,
        description="useful for when you need to ask with search",
        verbose=True
    )
]

In [170]:
prompt = hub.pull("hwchase17/self-ask-with-search")

In [171]:
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

In [172]:
self_ask_agent = create_self_ask_with_search_agent(llm,tools,prompt)

In [173]:
agent_executor = AgentExecutor(agent=agent,tools=tools,verbose=True)

In [174]:
agent_executor.invoke({"input":"How many moons does Jupiter have?"})



> Entering new AgentExecutor chain...
Thought: This question is about Jupiter's moons, not employee information. I can't answer this with the given tools. 
Final Answer: I cannot answer this question. 




> Finished chain.


{'input': 'How many moons does Jupiter have?',
 'output': 'I cannot answer this question.'}